In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/imdb-dataset/title.basics.tsv
/kaggle/input/imdb-dataset/title.principals.tsv
/kaggle/input/imdb-dataset/title.ratings.tsv
/kaggle/input/imdb-dataset/name.basics.tsv
/kaggle/input/imdb-dataset/title.crew.tsv
/kaggle/input/movielens/users.dat
/kaggle/input/movielens/ratings.dat
/kaggle/input/movielens/README
/kaggle/input/movielens/movies.dat


In [3]:
from scipy.sparse import csr_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

2024-07-07 17:42:56.775151: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-07 17:42:56.775263: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-07 17:42:56.891471: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
# # MovieLens datasets

mlens_movies_dtypes = {
    "movieId": "int64",
    "title": "string",
    "genres": "string"
}

mlens_ratings_dtypes = {
    "userId": "int64",
    "movieId": "int64",
    "rating": "int64",
    "timestamp": "int64"
}


mlens_movies_df = pd.read_csv("/kaggle/input/movielens/movies.dat", sep='::', header=None, engine='python', dtype=mlens_movies_dtypes, na_values="\\N", encoding='latin-1')
mlens_ratings_df = pd.read_csv("/kaggle/input/movielens/ratings.dat", sep='::', header=None, engine='python', dtype=mlens_ratings_dtypes, na_values="\\N")
mlens_movies_df.rename(columns={0: 'movieId', 1: 'title', 2:'genres'}, inplace=True)
mlens_ratings_df.rename(columns={0: 'userId', 1:'movieId', 2:'rating', 3:'timestamp'}, inplace=True)

In [5]:
# import datasets
# IMDb datasets
imdb_titles_dtypes = {
    "tconst": "string",
    "titleType": "string",
    "primaryTitle": "string",
    "originalTitle": "string",
    "isAdult": "string",
    "startYear": "Int64",  # using Int64 to handle NaNs
    "endYear": "Int64",    # using Int64 to handle NaNs
    "runtimeMinutes": "string",  # using Int64 to handle NaNs
    "genres": "string"
}
imdb_ratings_dtypes = {
    "tconst": "string",
    "averageRating": "float64",
    "numVotes": "int64"
}

# IMDb datasets
imdb_titles_df = pd.read_csv("/kaggle/input/imdb-dataset/title.basics.tsv", sep="\t", header=0, dtype=imdb_titles_dtypes, na_values="\\N")
imdb_ratings_df = pd.read_csv("/kaggle/input/imdb-dataset/title.ratings.tsv", sep="\t", header=0, dtype=imdb_ratings_dtypes, na_values="\\N")


In [6]:
imdb_titles_df

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,<NA>,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,<NA>,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,<NA>,5,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,<NA>,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,<NA>,1,"Comedy,Short"
...,...,...,...,...,...,...,...,...,...
10900922,tt9916848,tvEpisode,Episode #3.17,Episode #3.17,0,2009,<NA>,<NA>,"Action,Drama,Family"
10900923,tt9916850,tvEpisode,Episode #3.19,Episode #3.19,0,2010,<NA>,<NA>,"Action,Drama,Family"
10900924,tt9916852,tvEpisode,Episode #3.20,Episode #3.20,0,2010,<NA>,<NA>,"Action,Drama,Family"
10900925,tt9916856,short,The Wind,The Wind,0,2015,<NA>,27,Short


In [7]:
# removing rows with error on the columns (shift of the values to the left because originalTitle was put with primaryTitle so everything is shifted)
imdb_titles_df = imdb_titles_df[imdb_titles_df['genres'].notna()]
imdb_titles_df

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,<NA>,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,<NA>,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,<NA>,5,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,<NA>,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,<NA>,1,"Comedy,Short"
...,...,...,...,...,...,...,...,...,...
10900922,tt9916848,tvEpisode,Episode #3.17,Episode #3.17,0,2009,<NA>,<NA>,"Action,Drama,Family"
10900923,tt9916850,tvEpisode,Episode #3.19,Episode #3.19,0,2010,<NA>,<NA>,"Action,Drama,Family"
10900924,tt9916852,tvEpisode,Episode #3.20,Episode #3.20,0,2010,<NA>,<NA>,"Action,Drama,Family"
10900925,tt9916856,short,The Wind,The Wind,0,2015,<NA>,27,Short


In [8]:
# checking all titleTypes
imdb_titles_df['titleType'].unique()

<StringArray>
[       'short',        'movie',      'tvShort',      'tvMovie',
     'tvSeries',    'tvEpisode', 'tvMiniSeries',    'tvSpecial',
        'video',    'videoGame']
Length: 10, dtype: string

In [9]:
# keeping only movies / tvMovies
imdb_titles_df_movies = imdb_titles_df[imdb_titles_df['titleType'].str.contains('ovie')] 
imdb_titles_df_movies

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,<NA>,45,Romance
144,tt0000147,movie,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,0,1897,<NA>,100,"Documentary,News,Sport"
570,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,<NA>,70,"Action,Adventure,Biography"
587,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0,1907,<NA>,90,Drama
610,tt0000615,movie,Robbery Under Arms,Robbery Under Arms,0,1907,<NA>,<NA>,Drama
...,...,...,...,...,...,...,...,...,...
10900845,tt9916680,movie,De la ilusión al desconcierto: cine colombiano...,De la ilusión al desconcierto: cine colombiano...,0,2007,<NA>,100,Documentary
10900850,tt9916692,tvMovie,Teatroteka: Czlowiek bez twarzy,Teatroteka: Czlowiek bez twarzy,0,2015,<NA>,66,Drama
10900857,tt9916706,movie,Dankyavar Danka,Dankyavar Danka,0,2013,<NA>,<NA>,Comedy
10900867,tt9916730,movie,6 Gunn,6 Gunn,0,2017,<NA>,116,Drama


In [10]:
# removing na values and casting to right type
imdb_titles_df_clean = imdb_titles_df_movies.drop(columns=['endYear', 'originalTitle'])
imdb_titles_df_clean = imdb_titles_df_clean[imdb_titles_df_clean['startYear'].notna()]
imdb_titles_df_clean = imdb_titles_df_clean[imdb_titles_df_clean['runtimeMinutes'].notna()]
imdb_titles_df_clean['isAdult'] = imdb_titles_df_clean['isAdult'].astype(int)
imdb_titles_df_clean = imdb_titles_df_clean[imdb_titles_df_clean['isAdult'] <= 1]
imdb_titles_df_clean['isAdult'] = imdb_titles_df_clean['isAdult'] == 1
imdb_titles_df_clean['runtineMinutes'] = imdb_titles_df_clean['runtimeMinutes'].astype(int)
imdb_titles_df_clean['startYear'] = imdb_titles_df_clean['startYear'].astype(int)
imdb_titles_df_clean

,tconst,titleType,primaryTitle,isAdult,startYear,runtimeMinutes,genres,runtineMinutes
8,tt0000009,movie,Miss Jerry,False,1894,45,Romance,45
144,tt0000147,movie,The Corbett-Fitzsimmons Fight,False,1897,100,"Documentary,News,Sport",100
570,tt0000574,movie,The Story of the Kelly Gang,False,1906,70,"Action,Adventure,Biography",70
587,tt0000591,movie,The Prodigal Son,False,1907,90,Drama,90
672,tt0000679,movie,The Fairylogue and Radio-Plays,False,1908,120,"Adventure,Fantasy",120
...,...,...,...,...,...,...,...,...
10900818,tt9916622,movie,Rodolpho Teóphilo - O Legado de um Pioneiro,False,2015,57,Documentary,57
10900845,tt9916680,movie,De la ilusión al desconcierto: cine colombiano...,False,2007,100,Documentary,100
10900850,tt9916692,tvMovie,Teatroteka: Czlowiek bez twarzy,False,2015,66,Drama,66
10900867,tt9916730,movie,6 Gunn,False,2017,116,Drama,116


In [11]:
# two different movies with same title -> let's keep them
imdb_titles_df_clean[imdb_titles_df_clean['primaryTitle'] == 'Bohemios']

,tconst,titleType,primaryTitle,isAdult,startYear,runtimeMinutes,genres,runtineMinutes
30557,tt0031112,movie,Bohemios,False,1939,80,Musical,80
215673,tt0225200,movie,Bohemios,False,1935,62,Drama,62


In [12]:
imdb_ratings_df

,tconst,averageRating,numVotes
0,tt0000001,5.7,2061
1,tt0000002,5.6,278
2,tt0000003,6.5,2028
3,tt0000004,5.4,179
4,tt0000005,6.2,2796
...,...,...,...
1452482,tt9916730,7.0,12
1452483,tt9916766,7.1,23
1452484,tt9916778,7.2,37
1452485,tt9916840,7.2,10


In [13]:
len(imdb_ratings_df), len(imdb_ratings_df['numVotes'].notnull()),len(imdb_ratings_df['numVotes'] >= 50)

(1452487, 1452487, 1452487)

In [14]:
# merging the movies and their ratings info
imdb_df = pd.merge(imdb_titles_df_clean, imdb_ratings_df, how='inner', left_on='tconst', right_on='tconst')
imdb_df

,tconst,titleType,primaryTitle,isAdult,startYear,runtimeMinutes,genres,runtineMinutes,averageRating,numVotes
0,tt0000009,movie,Miss Jerry,False,1894,45,Romance,45,5.4,211
1,tt0000147,movie,The Corbett-Fitzsimmons Fight,False,1897,100,"Documentary,News,Sport",100,5.2,517
2,tt0000574,movie,The Story of the Kelly Gang,False,1906,70,"Action,Adventure,Biography",70,6.0,912
3,tt0000591,movie,The Prodigal Son,False,1907,90,Drama,90,5.6,26
4,tt0000679,movie,The Fairylogue and Radio-Plays,False,1908,120,"Adventure,Fantasy",120,5.3,74
...,...,...,...,...,...,...,...,...,...,...
319807,tt9916192,tvMovie,Danielle Darrieux: Il est poli d'être gai!,False,2019,53,Biography,53,7.3,18
319808,tt9916270,movie,Il talento del calabrone,False,2020,84,Thriller,84,5.8,1489
319809,tt9916362,movie,Coven,False,2020,92,"Drama,History",92,6.4,5844
319810,tt9916538,movie,Kuambil Lagi Hatiku,False,2019,123,Drama,123,8.3,9


In [15]:
# ensuring the same format for the title in order to merge the datasets
imdb_df['primaryTitle'] = imdb_df['primaryTitle'].str.lower()
imdb_df

,tconst,titleType,primaryTitle,isAdult,startYear,runtimeMinutes,genres,runtineMinutes,averageRating,numVotes
0,tt0000009,movie,miss jerry,False,1894,45,Romance,45,5.4,211
1,tt0000147,movie,the corbett-fitzsimmons fight,False,1897,100,"Documentary,News,Sport",100,5.2,517
2,tt0000574,movie,the story of the kelly gang,False,1906,70,"Action,Adventure,Biography",70,6.0,912
3,tt0000591,movie,the prodigal son,False,1907,90,Drama,90,5.6,26
4,tt0000679,movie,the fairylogue and radio-plays,False,1908,120,"Adventure,Fantasy",120,5.3,74
...,...,...,...,...,...,...,...,...,...,...
319807,tt9916192,tvMovie,danielle darrieux: il est poli d'être gai!,False,2019,53,Biography,53,7.3,18
319808,tt9916270,movie,il talento del calabrone,False,2020,84,Thriller,84,5.8,1489
319809,tt9916362,movie,coven,False,2020,92,"Drama,History",92,6.4,5844
319810,tt9916538,movie,kuambil lagi hatiku,False,2019,123,Drama,123,8.3,9


## MOVIELENS

In [16]:
mlens_movies_df

,movieId,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [17]:
# ensuring the same format as the imdb dataset and extracting the startYear info
mlens_movies_df['startYear'] = mlens_movies_df['title'].str[-5:-1].astype(int)
mlens_movies_df['title'] = mlens_movies_df['title'].str[:-7]
mlens_movies_df['title'] = mlens_movies_df['title'].str.lower()
mlens_movies_df

,movieId,title,genres,startYear
0,1,toy story,Animation|Children's|Comedy,1995
1,2,jumanji,Adventure|Children's|Fantasy,1995
2,3,grumpier old men,Comedy|Romance,1995
3,4,waiting to exhale,Comedy|Drama,1995
4,5,father of the bride part ii,Comedy,1995
...,...,...,...,...
3878,3948,meet the parents,Comedy,2000
3879,3949,requiem for a dream,Drama,2000
3880,3950,tigerland,Drama,2000
3881,3951,two family house,Drama,2000


In [18]:
mlens_ratings_df

,userId,movieId,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [19]:
# dropping useless column timestamp
mlens_ratings_df = mlens_ratings_df.drop(columns='timestamp')
mlens_ratings_df

,userId,movieId,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
...,...,...,...
1000204,6040,1091,1
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4


In [20]:
ratings_df = mlens_ratings_df
ratings_df

,userId,movieId,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
...,...,...,...
1000204,6040,1091,1
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4


## MERGING

In [21]:
# merging based on title
movies_df = pd.merge(mlens_movies_df, imdb_df, how='inner', left_on=['title'], right_on=['primaryTitle'])
movies_df

,movieId,title,genres_x,startYear_x,tconst,titleType,primaryTitle,isAdult,startYear_y,runtimeMinutes,genres_y,runtineMinutes,averageRating,numVotes
0,1,toy story,Animation|Children's|Comedy,1995,tt0114709,movie,toy story,False,1995,81,"Adventure,Animation,Comedy",81,8.3,1075934
1,2,jumanji,Adventure|Children's|Fantasy,1995,tt0113497,movie,jumanji,False,1995,104,"Adventure,Comedy,Family",104,7.1,379991
2,3,grumpier old men,Comedy|Romance,1995,tt0113228,movie,grumpier old men,False,1995,101,"Comedy,Romance",101,6.6,29870
3,4,waiting to exhale,Comedy|Drama,1995,tt0114885,movie,waiting to exhale,False,1995,124,"Comedy,Drama,Romance",124,6.0,12307
4,5,father of the bride part ii,Comedy,1995,tt0113041,movie,father of the bride part ii,False,1995,106,"Comedy,Family,Romance",106,6.1,41935
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5278,3948,meet the parents,Comedy,2000,tt0212338,movie,meet the parents,False,2000,108,"Comedy,Romance",108,7.0,357316
5279,3949,requiem for a dream,Drama,2000,tt0180093,movie,requiem for a dream,False,2000,102,Drama,102,8.3,904118
5280,3950,tigerland,Drama,2000,tt0170691,movie,tigerland,False,2000,101,"Drama,War",101,6.9,43498
5281,3950,tigerland,Drama,2000,tt7913478,movie,tigerland,False,2019,91,Documentary,91,6.9,72


In [22]:
# cleaning the merged dataset
movies_df = movies_df.drop(columns=['runtimeMinutes', 'primaryTitle'])
movies_df['genres_x'] = movies_df['genres_x'].str.split("|")
movies_df['genres_y'] = movies_df['genres_y'].str.split(",")
movies_df['genres'] = movies_df['genres_x'] + movies_df['genres_y']
movies_df = movies_df.drop(columns=['genres_x', 'genres_y'])
movies_df

,movieId,title,startYear_x,tconst,titleType,isAdult,startYear_y,runtineMinutes,averageRating,numVotes,genres
0,1,toy story,1995,tt0114709,movie,False,1995,81,8.3,1075934,"[Animation, Children's, Comedy, Adventure, Ani..."
1,2,jumanji,1995,tt0113497,movie,False,1995,104,7.1,379991,"[Adventure, Children's, Fantasy, Adventure, Co..."
2,3,grumpier old men,1995,tt0113228,movie,False,1995,101,6.6,29870,"[Comedy, Romance, Comedy, Romance]"
3,4,waiting to exhale,1995,tt0114885,movie,False,1995,124,6.0,12307,"[Comedy, Drama, Comedy, Drama, Romance]"
4,5,father of the bride part ii,1995,tt0113041,movie,False,1995,106,6.1,41935,"[Comedy, Comedy, Family, Romance]"
...,...,...,...,...,...,...,...,...,...,...,...
5278,3948,meet the parents,2000,tt0212338,movie,False,2000,108,7.0,357316,"[Comedy, Comedy, Romance]"
5279,3949,requiem for a dream,2000,tt0180093,movie,False,2000,102,8.3,904118,"[Drama, Drama]"
5280,3950,tigerland,2000,tt0170691,movie,False,2000,101,6.9,43498,"[Drama, Drama, War]"
5281,3950,tigerland,2000,tt7913478,movie,False,2019,91,6.9,72,"[Drama, Documentary]"


In [23]:
movies_df = movies_df[movies_df['startYear_x'] == movies_df['startYear_y']]
movies_df = movies_df.drop(columns=['startYear_y', 'tconst']).rename(columns={'startYear_x': 'startYear'})
movies_df

,movieId,title,startYear,titleType,isAdult,runtineMinutes,averageRating,numVotes,genres
0,1,toy story,1995,movie,False,81,8.3,1075934,"[Animation, Children's, Comedy, Adventure, Ani..."
1,2,jumanji,1995,movie,False,104,7.1,379991,"[Adventure, Children's, Fantasy, Adventure, Co..."
2,3,grumpier old men,1995,movie,False,101,6.6,29870,"[Comedy, Romance, Comedy, Romance]"
3,4,waiting to exhale,1995,movie,False,124,6.0,12307,"[Comedy, Drama, Comedy, Drama, Romance]"
4,5,father of the bride part ii,1995,movie,False,106,6.1,41935,"[Comedy, Comedy, Family, Romance]"
...,...,...,...,...,...,...,...,...,...
5275,3947,get carter,1971,movie,False,112,7.3,37001,"[Thriller, Action, Crime, Thriller]"
5278,3948,meet the parents,2000,movie,False,108,7.0,357316,"[Comedy, Comedy, Romance]"
5279,3949,requiem for a dream,2000,movie,False,102,8.3,904118,"[Drama, Drama]"
5280,3950,tigerland,2000,movie,False,101,6.9,43498,"[Drama, Drama, War]"


In [24]:
movies_df_save = movies_df

In [25]:
movies_df_save['genres'] = movies_df_save['genres'].apply(lambda x: list(set(x)))
movies_df_exploded = movies_df.explode('genres')
df_onehot = pd.get_dummies(movies_df_exploded['genres'])
df_onehot = df_onehot.groupby(df_onehot.index).sum()
movies_df_final = movies_df_save.drop(columns='genres').join(df_onehot).groupby(['movieId', 'title', 'startYear', 'titleType', 'isAdult', 'runtineMinutes', 'averageRating', 'numVotes'], as_index=False).sum()
movies_df_final

,movieId,title,startYear,titleType,isAdult,runtineMinutes,averageRating,numVotes,Action,Adventure,...,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
0,1,toy story,1995,movie,False,81,8.3,1075934,0,1,...,0,0,0,0,0,0,0,0,0,0
1,2,jumanji,1995,movie,False,104,7.1,379991,0,1,...,0,0,0,0,0,0,0,0,0,0
2,3,grumpier old men,1995,movie,False,101,6.6,29870,0,0,...,0,0,0,0,1,0,0,0,0,0
3,4,waiting to exhale,1995,movie,False,124,6.0,12307,0,0,...,0,0,0,0,1,0,0,0,0,0
4,5,father of the bride part ii,1995,movie,False,106,6.1,41935,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2426,3947,get carter,1971,movie,False,112,7.3,37001,1,0,...,0,0,0,0,0,0,0,1,0,0
2427,3948,meet the parents,2000,movie,False,108,7.0,357316,0,0,...,0,0,0,0,1,0,0,0,0,0
2428,3949,requiem for a dream,2000,movie,False,102,8.3,904118,0,0,...,0,0,0,0,0,0,0,0,0,0
2429,3950,tigerland,2000,movie,False,101,6.9,43498,0,0,...,0,0,0,0,0,0,0,0,1,0


In [26]:
# keeping only the ratings of the movies present in movies_df_final
filtered_ratings_df = ratings_df[ratings_df['movieId'].isin(movies_df_final['movieId'])]
filtered_ratings_df

,userId,movieId,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
6,1,1287,5
...,...,...,...
1000203,6040,1090,3
1000204,6040,1091,1
1000206,6040,562,5
1000207,6040,1096,4


In [27]:
ratings_matrix_df = filtered_ratings_df.pivot(index='movieId', columns='userId', values='rating').fillna(0)
ratings_matrix_df

userId,1,2,3,4,5,6,7,8,9,10,...,6031,6032,6033,6034,6035,6036,6037,6038,6039,6040
movieId,,,,,,,,,,,,,,,,,,,,,
1,5.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,5.0,5.0,...,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,3.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3947,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3948,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3949,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
# Collaborative filtering
def cofi_cost_function(X, W, b, Y, R, lambda_):
    predictions = tf.matmul(X, tf.transpose(W)) + b
    error = (predictions - Y) * R  # Element-wise multiplication by R
    cost = 0.5 * tf.reduce_sum(tf.square(error))
    regularization = (lambda_ / 2) * (tf.reduce_sum(tf.square(X)) + tf.reduce_sum(tf.square(W)))
    return cost + regularization

In [53]:
Y = ratings_matrix_df.values
R = (Y != 0).astype(int)

Ymean = np.mean(Y, axis=1, keepdims=True)
Ynorm = Y - Ymean

num_movies, num_users = Y.shape
num_features = 20

tf.random.set_seed(42)
X = tf.Variable(tf.random.normal((num_movies, num_features), dtype=tf.float64))
W = tf.Variable(tf.random.normal((num_users, num_features), dtype=tf.float64))
b = tf.Variable(tf.zeros((1, num_users), dtype=tf.float64))

iterations = 100
lambda_ = 2
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-1)

for iter in range(iterations):
    with tf.GradientTape() as tape:
        cost_value = cofi_cost_function(X, W, b, Ynorm, R, lambda_)
    grads = tape.gradient(cost_value, [X, W, b])
    optimizer.apply_gradients(zip(grads, [X, W, b]))
    if iter % 10 == 0:
        print(f"Training loss at iteration {iter}: {cost_value:.2f}")

p = np.matmul(X.numpy(), W.numpy().T) + b.numpy()
predictions = p + Ymean

Training loss at iteration 0: 10739941.49
Training loss at iteration 10: 1497036.05
Training loss at iteration 20: 495662.25
Training loss at iteration 30: 344273.59
Training loss at iteration 40: 289016.70
Training loss at iteration 50: 258342.50
Training loss at iteration 60: 240490.70
Training loss at iteration 70: 228031.24
Training loss at iteration 80: 218861.56
Training loss at iteration 90: 211968.15


In [31]:
movies_df_final.columns

Index(['movieId', 'title', 'startYear', 'titleType', 'isAdult',
       'runtineMinutes', 'averageRating', 'numVotes', 'Action', 'Adventure',
       'Animation', 'Biography', 'Children's', 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Family', 'Fantasy', 'Film-Noir', 'History',
       'Horror', 'Music', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Sport',
       'Thriller', 'War', 'Western'],
      dtype='object')

In [32]:
genres_list = ['Action', 'Adventure',
       'Animation', 'Biography', 'Children\'s', 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Family', 'Fantasy', 'Film-Noir', 'History',
       'Horror', 'Music', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Sport',
       'Thriller', 'War', 'Western']

# prepare movie features
movies_features_df = movies_df_final[['movieId', 'isAdult', 'averageRating', 'numVotes'] + genres_list]
movies_features_df = movies_features_df[movies_features_df['movieId'].isin(filtered_ratings_df['movieId'])]

# scale numeric values
scaler = MinMaxScaler()
numeric_features = ['averageRating', 'numVotes']
movies_features_df[numeric_features] = scaler.fit_transform(movies_features_df[numeric_features])

# select features
feature_columns = ['isAdult', 'averageRating', 'numVotes'] + genres_list
movie_features_matrix = movies_features_df[feature_columns].values

movie_id_to_title = dict(zip(movies_df_final['movieId'], movies_df_final['title']))
movies_features_df[feature_columns]

,isAdult,averageRating,numVotes,Action,Adventure,Animation,Biography,Children's,Comedy,Crime,...,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
0,False,0.900000,0.458966,0,1,1,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,False,0.728571,0.162092,0,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,False,0.657143,0.012739,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
3,False,0.571429,0.005247,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
4,False,0.585714,0.017886,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2426,False,0.757143,0.015781,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
2427,False,0.714286,0.152420,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
2428,False,0.900000,0.385673,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2429,False,0.700000,0.018552,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [54]:
def recommand_users(user_id_1, user_id_2, predicitons, matrix, movies_df, top_n=10):
    movie_ids = movies_df['movieId'].values
    
    # Get already rated movies for both users
    already_rated_user_1 = filtered_ratings_df[filtered_ratings_df['userId'] == user_id_1]['movieId'].values
    already_rated_user_2 = filtered_ratings_df[filtered_ratings_df['userId'] == user_id_2]['movieId'].values

    # Combined already rated movies
    already_rated = np.union1d(already_rated_user_1, already_rated_user_2)

    # Get predicitions for users
    user_predictions_1 = predictions[:, user_id_1]
    user_predictions_2 = predictions[:, user_id_2]
    average_predictions = (user_predictions_1 + user_predictions_2) / 2
    
    # Use movie features to adjust the average predictions
#     hybrid_scores = average_predictions + movie_features_matrix.dot(np.random.rand(movie_features_matrix.shape[1]))

    # Sort indices based on the average predictions in descending order
#     sorted_indices = np.argsort(hybrid_scores)[::-1]
    sorted_indices = np.argsort(average_predictions)[::-1]

    print(f"Top recommendations for users {user_id_1} and {user_id_2}")
    recommendations_count = 0
    for idx in sorted_indices:
        i = matrix.columns[idx]
        movie_id = movie_ids[i]
        if movie_id not in already_rated:
            movie_title = movies_df_final[movies_df_final['movieId'] == movie_id]['title'].values[0]
            print(f"Predicting rating {average_predictions[idx]:.2f} for movie {movie_id}: {movie_title}")
            recommendations_count += 1
            if recommendations_count >= 10:
                break
        else:
            original_rating_1 = matrix[user_id_1][movie_id]
            original_rating_2 = matrix[user_id_2][movie_id]
            print(f"Original ratings: {original_rating_1} {original_rating_2}, predicted rating: {average_predictions[idx]:.2f} for movie {movie_id}: {movie_title}")

In [55]:
recommand_users(4, 5, predictions, ratings_matrix_df, movies_df_final)

Top recommendations for users 4 and 5
Predicting rating 4.25 for movie 2219: murder!
Predicting rating 4.25 for movie 885: bogus
Predicting rating 4.13 for movie 2964: julien donkey-boy
Predicting rating 4.08 for movie 2956: someone to watch over me
Predicting rating 4.07 for movie 3343: and god created woman
Predicting rating 4.05 for movie 902: breakfast at tiffany's
Predicting rating 3.99 for movie 857: crows and sparrows
Predicting rating 3.98 for movie 2035: blackbeard's ghost
Predicting rating 3.97 for movie 3461: lord of the flies
Predicting rating 3.97 for movie 2949: dr. no


## Evaluation

In [73]:
difference_matrix = np.abs(ratings_matrix_df, predictions)
difference_matrix = difference_matrix[(difference_matrix != 0).any(axis=1)]
difference_matrix

userId,1,2,3,4,5,6,7,8,9,10,...,6031,6032,6033,6034,6035,6036,6037,6038,6039,6040
movieId,,,,,,,,,,,,,,,,,,,,,
1,5.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,5.0,5.0,...,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,3.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3947,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3948,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3949,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [74]:
max_diff, min_diff, mean_diff = difference_matrix.mean(axis=1).max(), difference_matrix.mean(axis=1).min(), difference_matrix.mean(axis=1).mean()
max_index, min_index = difference_matrix.mean(axis=1).idxmax(), difference_matrix.mean(axis=1).idxmin()
print(f"The worst predictions were given to the user {max_index} with an average difference of {max_diff}")
print(f"The best predictions were given to the user {min_index} with an average difference of {mean_diff}")
print(f"In average the difference between the original ratings and the predicted one is : {mean_diff}")

The worst predictions were given to the user 2858 with an average difference of 2.4503311258278146
The best predictions were given to the user 133 with an average difference of 0.17632046864426695
In average the difference between the original ratings and the predicted one is : 0.17632046864426695
